In [1]:
import numpy as np
import pandas as pd
import networkx as nx
import plotly.io as pio
import plotly.express as px
import plotly.graph_objects as go
import igviz as ig
from node2vec import Node2Vec
from gensim.models import KeyedVectors
import seaborn as sns
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2


# import src.preprocess as pre
# import src.visualize as vis
# pio.renderers.default = "png"

from src import models, training
from torch_geometric.nn import GAE
import torch
import os
import genie3

/nethome/mbafna6/miniconda3/envs/CeSpGRN/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
num_cells = 1597

In [39]:
df = pd.read_csv("./data/scMultiSim_final/scmultisim_dataframe.csv")
df5 = df[df['Cell_Type'] == "2_1_5"].iloc[:,4:].values
df4 = df[df['Cell_Type'] == "2_1_4"].iloc[:,4:].values
df3 = df[df['Cell_Type'] == "2_1_3"].iloc[:,4:].values
df2 = df[df['Cell_Type'] == "2_1_2"].iloc[:,4:].values
df1 = df[df['Cell_Type'] == "2_1_1"].iloc[:,4:].values

countlist = [df1,df2,df3,df4,df5]
gene_names = df.columns[4:]


In [42]:
celltype_grns = []
for i,counts in enumerate(countlist):
    print(f"Cell Type {i+1} GRN, counts shape {counts.shape}")
    type_grn = genie3.GENIE3(counts, gene_names=gene_names, nthreads=8)
    np.save(f"./benchmark/genie3/scmultisim_final/CellType{i+1}GRN.npy", type_grn)
    celltype_grns.append(type_grn)

Cell Type 1 GRN, counts shape (240, 120)
Tree method: RF, K: sqrt, Number of trees: 1000 

Cell Type 2 GRN, counts shape (240, 120)
Tree method: RF, K: sqrt, Number of trees: 1000 

Cell Type 3 GRN, counts shape (240, 120)
Tree method: RF, K: sqrt, Number of trees: 1000 

Cell Type 4 GRN, counts shape (240, 120)
Tree method: RF, K: sqrt, Number of trees: 1000 

Cell Type 5 GRN, counts shape (240, 120)
Tree method: RF, K: sqrt, Number of trees: 1000 



In [46]:
gt_grn_path = "./data/scMultiSim_final/simulated/cellgrn/"

gt_grns = np.zeros((len(df),len(gene_names),60))

for i, filename in enumerate(os.listdir(gt_grn_path)):
    grn = pd.read_csv(os.path.join(gt_grn_path, filename)).values
    
    gt_grns[i, : ,:] = grn
    
gt_grns.shape, gt_grns
np.save("./data/scMultiSim_final/gt_grns.npy", gt_grns)

In [48]:
gt_grns = np.load("./data/scMultiSim_final/gt_grns.npy")
gt_grns.shape

(1200, 120, 60)

In [50]:
genie3_grns = np.zeros((len(df),len(gene_names),60))

for i, row in df.iterrows():
    type_num = row["Cell_Type"].split("_")[-1]
    celltype_grn = np.load(f"./benchmark/genie3/scmultisim_final/CellType{type_num}GRN.npy")[:,:60]
    genie3_grns[i,:,:] = celltype_grn
np.save("./benchmark/genie3/scmultisim_final/genie3_grns.npy", genie3_grns)
    

In [75]:

gt_grns = np.load("./data/scMultiSim_final/gt_grns.npy")
genie3_grns = np.load("./benchmark/genie3/scmultisim_final/genie3_grns.npy")
cespgrn_grns = np.load("./out/scmultisim_final/1_preprocessing_output/initial_grns.npy")
clarify_grns = np.load("./benchmark/genie3/clarify_grns.npy")

In [186]:
from sklearn.metrics import roc_auc_score, jaccard_score, accuracy_score, f1_score, precision_recall_curve, auc

total_genes = 1200 * 120 * 60

network_density = np.sum(gt_grns.flatten()>0) / total_genes

p, r, _ = precision_recall_curve(gt_grns.flatten()>0, genie3_grns.flatten())
network_density, auc(r,p) / network_density

# auc(r,p) 

(0.01853888888888889, 1.4029852482084098)

In [187]:
# total_genes = 1200 * 60 * 59
# network_density = np.sum(gt_grns[:,:45,:45].flatten()) / total_genes


p, r, _ = precision_recall_curve(gt_grns[:,:45,:45].flatten()>0, clarify_grns.flatten())
network_density, auc(r,p) / network_density


(0.01853888888888889, 1.474749717495629)

In [104]:

f1_score(gt_grns[:,:45,:45].flatten()>4,clarify_grns.flatten()>0, average="micro")

0.026814814814814812

In [106]:
inverted_gt = (gt_grns[:,:45,:45].flatten()>4) == 0

In [108]:
f1_score(inverted_gt,(clarify_grns.flatten()>0)==0, average="micro")

0.026814814814814812

In [110]:
roc_auc_score(gt_grns.flatten()>0, np.rand.permute(gt_grns.flatten()>0))

AttributeError: module 'numpy' has no attribute 'rand'